In [1]:
import tensorflow as tf
import data.imageReading as ir
from fairness import fairnessMetrics as fm
from model import baseModel as bm
from model import utils as utils
from model import biasMitigation as mit
from model import transferLearning as tl
from model import evaluation as ev

2022-11-26 11:33:12.651088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-26 11:33:12.824611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-26 11:33:12.824631: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-26 11:33:12.860236: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-26 11:33:13.835329: W tensorflow/stream_executor/platform/de

In [2]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

image_size = (224,224)
batch_size = 2
epochs = 60
(ds_train, ds_val, ds_test, count_classes) = ir.readData("../FairFace", image_size, batch_size, preprocess_input)

class_weight = mit.findClassWeights(ds_train)

base_model = bm.build_model()

utils.train_model(base_model, epochs, ds_train, ds_val, class_weight)

utils.saveModel(base_model, "base_model")

Delete old data folder: ../FairFace_split


Copying files: 97698 files [00:25, 3789.42 files/s]


Found 78158 images belonging to 2 classes.
Found 9769 images belonging to 2 classes.
Found 9771 images belonging to 2 classes.
Count classes: (36736, 41422, 4592, 5177, 4592, 5179)
Weight for class 0: 1.06
Weight for class 1: 0.94


2022-11-24 10:30:55.411630: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-24 10:30:55.411651: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-24 10:30:55.411669: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (christoph-XPS-13-9370): /proc/driver/nvidia/version does not exist
2022-11-24 10:30:55.411856: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


sc
Epoch 1/60


KeyboardInterrupt: 

In [3]:
utils.saveModel(base_model, "base_model.h5")

In [2]:
model = utils.loadModel("base_model.h5")

2022-11-26 11:33:17.858103: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-26 11:33:17.858121: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-26 11:33:17.858137: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (christoph-XPS-13-9370): /proc/driver/nvidia/version does not exist
2022-11-26 11:33:17.858287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
tl.freezeAllLayers(model)
tl.unfreezeLayers(model, -5)
#for l in model.layers:
#    print(l.name, l.trainable)
preprocess_input = tf.keras.applications.vgg16.preprocess_input

image_size = (224,224)
batch_size = 32
model.compile(loss="binary_crossentropy")

(ds_train, ds_val, ds_test, count_classes) = ir.readData("../museumFaces", image_size, batch_size, preprocess_input)

class_weight = mit.findClassWeights(ds_train)

epochs = 20
utils.train_model(model, epochs, ds_train, ds_val, class_weight)

Delete old data folder: ../museumFaces_split


Copying files: 4897 files [00:00, 9062.34 files/s]

Found 3916 images belonging to 2 classes.


Found 489 images belonging to 2 classes.
Found 492 images belonging to 2 classes.
Count classes: (2052, 1864, 256, 233, 258, 234)
Weight for class 0: 0.95
Weight for class 1: 1.05
sc
Epoch 1/20


KeyboardInterrupt: 

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test)
ev.testModelWithThresholdChange(model, ds_val, test_predict, test_labels, dir_name)

Testing Model
-------------
16/16 - 143s - 143s/epoch - 9s/step
Make folder...
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.5101626016260162
    
    True positives: 97
    False positives: 104
    
    True negatives: 154
    False negatives: 137
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.41452991452991456
    True negative rate tn/(tn+fp): 0.5968992248062015
    
    False negative rate fn/(tp+fn): 0.5854700854700855
    False positive rate fp/(tn+fp): 0.40310077519379844
    
    Positive predicted value tp/(tp+fp): 0.48258706467661694
    False discovery rate fp/(tp+fp): 0.5174129353233831
    
    Negative predicted value tn/(tn+fn): 0.5292096219931272
    False omission rate fn/(tn+fn): 0.47079037800687284
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -90
    Demographic parity tp+fp: 201
    
    Binary 